# LAB | Extractive Question Answering

This notebook demonstrates how Pinecone helps you build an extractive question-answering application. To build an extractive question-answering system, we need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A reader model to extract answers

We will use the SQuAD dataset, which consists of **questions** and **context** paragraphs containing question **answers**. We generate embeddings for the context passages using the retriever, index them in the vector database, and query with semantic search to retrieve the top k most relevant contexts containing potential answers to our question. We then use the reader model to extract the answers from the returned contexts.

Let's get started by installing the packages needed for notebook to run:

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY= os.getenv('PINECONE_API_KEY')

ModuleNotFoundError: No module named 'dotenv'

In [37]:
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY_Ironhack')
PINECONE_API_KEY= userdata.get('PINECONE_API_KEY')

# Install Dependencies

In [ ]:
!pip install -qU datasets pinecone-client sentence-transformers torch

     |████████████████████████████████| 365 kB 39.2 MB/s 
     |████████████████████████████████| 120 kB 74.9 MB/s 
     |████████████████████████████████| 115 kB 74.4 MB/s 
     |████████████████████████████████| 212 kB 77.0 MB/s 
     |████████████████████████████████| 127 kB 65.4 MB/s 
     |████████████████████████████████| 175 kB 24.9 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
     |████████████████████████████████| 269 kB 66.6 MB/s 
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 4.7 MB 61.6 MB/s 
     |████████████████████████████████| 1.3 MB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 59.9 MB/s 


# Load Dataset

Now let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [38]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()

In [39]:
# select only title and context column
df = df[["title", "context"]]
# drop rows containing duplicate context passages
df = df.dropna().drop_duplicates()
df

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
5,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
10,University_of_Notre_Dame,The university is the major seat of the Congre...
15,University_of_Notre_Dame,The College of Engineering was established in ...
20,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...
...,...,...
87574,Kathmandu,"Institute of Medicine, the central college of ..."
87579,Kathmandu,Football and Cricket are the most popular spor...
87584,Kathmandu,The total length of roads in Nepal is recorded...
87589,Kathmandu,The main international airport serving Kathman...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our context passages which we can retrieve using another vector (query vector). We first need to initialize our connection to Pinecone to create our vector index. For this, we need a free [API key]("https://app.pinecone.io/"), and then we initialize the connection like so:

In [40]:
from pinecone import Pinecone, ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'  # find next to API key in console
)


Now we create a new index called "question-answering" — we can name the index anything we want. We specify the metric type as "cosine" and dimension as 384 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

In [41]:
import time

index_name = 'question-answering' #None
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if the extractive-question-answering index exists
if index_name not in existing_indexes:
    # create the index if it does not exist
    #None
    pc.create_index(
        index_name,
        dimension=384,  #specify the metric type as "cosine" and dimension as 384
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to extractive-question-answering index we created
index = pc.Index(index_name)


In [42]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever.

In [43]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load the retriever model from huggingface model hub
retriever = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1', device=device)
retriever

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, context passage, etc.

In [44]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    end = min(i + batch_size, len(df))

    # extract batch
    batch = df.iloc[i:end]

    # generate embeddings for batch
    emb = retriever.encode(batch['context'].tolist())

    # get metadata
    meta = [{'title': title, 'context': context} for title, context in zip(batch['title'], batch['context'])]

    # create unique IDs
    ids = [str(j) for j in range(i, end)]

    # add all to upsert list
    to_upsert = [(id, emb.tolist(), m) for id, emb, m in zip(ids, emb, meta)]

    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 21056}},
 'total_vector_count': 21056}

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into a "question-answering" pipeline from HuggingFace transformers and feed it our questions and context passages individually. The model gives a prediction for each context we pass through the pipeline.

In [45]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task='question-answering', device=device)
reader

Now all the components we need are ready. Let's write some helper functions to execute our queries. The `get_context` function retrieves the context embeddings containing answers to our question from the Pinecone index, and the `extract_answer` function extracts the answers from these context passages.

In [46]:
def get_context(question, top_k):
    # Generate embeddings for the question
    xq = retriever.encode([question])
    xq = xq[0].tolist()

    # Search Pinecone index for context passages related to the question
    xc = index.query(vector=[xq], top_k=top_k, include_metadata=True)

    # Extract the context passages from Pinecone search results
    c = [match['metadata']['context'] for match in xc['matches']]

    return c


In [47]:
from pprint import pprint

# extracts answer from the context passage
def extract_answer(question, context):
    results = []
    for c in context:
        # Feed the reader model the question and context to extract answers
        answer = reader(question=question, context=c)
        # Add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)

    # Sort the result based on the score from reader model
    sorted_result = sorted(results, key=lambda x: x['score'], reverse=True)
    pprint(sorted_result)
    return sorted_result

In [28]:
question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k = 1)
context

['Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of natural gas (in 2013), which makes Egypt as the largest oil producer not member of the Organization of the Petroleum Exporting Countries (OPEC) and the second-largest dry natural gas producer in Africa. In 2013, Egypt was the largest consumer of oil and natural gas in Africa, as more than 20% of total oil consumption and more than 40% of total dry natural gas consumption in Africa. Also, Egypt possesses the largest oil refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is currently planning to build its first nuclear power plant in El Dabaa city, northern Egypt.']

As we can see, the retiever is working fine and gets us the context passage that contains the answer to our question. Now let's use the reader to extract the exact answer from the context passage.

In [ ]:
extract_answer(question, context)

[{'answer': '691,000 bbl/d',
  'context': 'Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of '
             'natural gas (in 2013), which makes Egypt as the largest oil '
             'producer not member of the Organization of the Petroleum '
             'Exporting Countries (OPEC) and the second-largest dry natural '
             'gas producer in Africa. In 2013, Egypt was the largest consumer '
             'of oil and natural gas in Africa, as more than 20% of total oil '
             'consumption and more than 40% of total dry natural gas '
             'consumption in Africa. Also, Egypt possesses the largest oil '
             'refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is '
             'currently planning to build its first nuclear power plant in El '
             'Dabaa city, northern Egypt.',
  'end': 33,
  'score': 0.9999852180480957,
  'start': 20}]


In [48]:
extract_answer(question, context)

[{'answer': 'Earth,',
  'context': 'The first step was witnessed by at least one-fifth of the '
             'population of Earth, or about 723 million people. His first '
             "words when he stepped off the LM's landing footpad were, "
             '"That\'s one small step for [a] man, one giant leap for '
             'mankind." Aldrin joined him on the surface almost 20 minutes '
             'later. Altogether, they spent just under two and one-quarter '
             'hours outside their craft. The next day, they performed the '
             'first launch from another celestial body, and rendezvoused back '
             'with Columbia.',
  'end': 78,
  'score': 3.191796549284476e-10,
  'start': 72},
 {'answer': '9:56 pm CDT July 20',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
         

[{'score': 3.191796549284476e-10,
  'start': 72,
  'end': 78,
  'answer': 'Earth,',
  'context': 'The first step was witnessed by at least one-fifth of the population of Earth, or about 723 million people. His first words when he stepped off the LM\'s landing footpad were, "That\'s one small step for [a] man, one giant leap for mankind." Aldrin joined him on the surface almost 20 minutes later. Altogether, they spent just under two and one-quarter hours outside their craft. The next day, they performed the first launch from another celestial body, and rendezvoused back with Columbia.'},
 {'score': 5.775184475607931e-13,
  'start': 688,
  'end': 707,
  'answer': '9:56 pm CDT July 20',
  'context': 'The trip to the Moon took just over three days. After achieving orbit, Armstrong and Aldrin transferred into the Lunar Module, named Eagle, and after a landing gear inspection by Collins remaining in the Command/Service Module Columbia, began their descent. After overcoming several computer o

The reader model predicted with 99% accuracy the correct answer *691,000 bbl/d* as seen from the context passage. Let's run few more queries.

In [49]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Hurley and Chen',
  'context': 'According to a story that has often been repeated in the media, '
             'Hurley and Chen developed the idea for YouTube during the early '
             'months of 2005, after they had experienced difficulty sharing '
             "videos that had been shot at a dinner party at Chen's apartment "
             'in San Francisco. Karim did not attend the party and denied that '
             'it had occurred, but Chen commented that the idea that YouTube '
             'was founded after a dinner party "was probably very strengthened '
             'by marketing ideas around creating a story that was very '
             'digestible".',
  'end': 79,
  'score': 0.9999276399612427,
  'start': 64}]


[{'score': 0.9999276399612427,
  'start': 64,
  'end': 79,
  'answer': 'Hurley and Chen',
  'context': 'According to a story that has often been repeated in the media, Hurley and Chen developed the idea for YouTube during the early months of 2005, after they had experienced difficulty sharing videos that had been shot at a dinner party at Chen\'s apartment in San Francisco. Karim did not attend the party and denied that it had occurred, but Chen commented that the idea that YouTube was founded after a dinner party "was probably very strengthened by marketing ideas around creating a story that was very digestible".'}]

In [33]:
question = "What is Albert Eistein famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'his theories of special relativity and general relativity',
  'context': 'Albert Einstein is known for his theories of special relativity '
             'and general relativity. He also made important contributions to '
             'statistical mechanics, especially his mathematical treatment of '
             'Brownian motion, his resolution of the paradox of specific '
             'heats, and his connection of fluctuations and dissipation. '
             'Despite his reservations about its interpretation, Einstein also '
             'made contributions to quantum mechanics and, indirectly, quantum '
             'field theory, primarily through his theoretical studies of the '
             'photon.',
  'end': 86,
  'score': 0.9500371217727661,
  'start': 29}]


[{'score': 0.9500371217727661,
  'start': 29,
  'end': 86,
  'answer': 'his theories of special relativity and general relativity',
  'context': 'Albert Einstein is known for his theories of special relativity and general relativity. He also made important contributions to statistical mechanics, especially his mathematical treatment of Brownian motion, his resolution of the paradox of specific heats, and his connection of fluctuations and dissipation. Despite his reservations about its interpretation, Einstein also made contributions to quantum mechanics and, indirectly, quantum field theory, primarily through his theoretical studies of the photon.'}]

Let's run another question. This time for top 3 context passages from the retriever.

In [34]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
             'remaining in the Command/Service Module Columbia, began their '
             'descent. After overcoming several computer overload alarms '
             'caused by an antenna switch left in the wrong position, and a '
             'slight downrange error, Armstrong took over manual flight '
             'control at about 180 meters (590 ft), and guided the Lunar '
             'Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 '
             '(3:17:04 pm CDT). The first humans on the Moon would wait '
             'another six hours before they ventured out of their craft. At '
             '02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the '
             'first human to set foot on the Moon.',

[{'score': 0.9998037815093994,
  'start': 71,
  'end': 80,
  'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving orbit, Armstrong and Aldrin transferred into the Lunar Module, named Eagle, and after a landing gear inspection by Collins remaining in the Command/Service Module Columbia, began their descent. After overcoming several computer overload alarms caused by an antenna switch left in the wrong position, and a slight downrange error, Armstrong took over manual flight control at about 180 meters (590 ft), and guided the Lunar Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 (3:17:04 pm CDT). The first humans on the Moon would wait another six hours before they ventured out of their craft. At 02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the first human to set foot on the Moon.'},
 {'score': 0.9998037815093994,
  'start': 71,
  'end': 80,
  'answer': 'Armstrong',
  'context': 'The trip to the Moon took just ov

The result looks pretty good.

In [35]:
pc.delete_index(index_name)

### Add a few more questions. What did you observe?

In [51]:
question = "Which country has the biggest area?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'United States',
  'context': 'By total area (water as well as land), the United States is '
             "either slightly larger or smaller than the People's Republic of "
             "China, making it the world's third or fourth largest country. "
             'China and the United States are smaller than Russia and Canada '
             'in total area, but are larger than Brazil. By land area only '
             "(exclusive of waters), the United States is the world's third "
             'largest country, after Russia and China, with Canada in fourth. '
             'Whether the US or China is the third largest country by total '
             "area depends on two factors: (1) The validity of China's claim "
             'on Aksai Chin and Trans-Karakoram Tract. Both these territories '
             'are also claimed by India, so are not counted; and (2) How US '
             'calculates its own surface area. Since the initial publishing of '
             'the World Fac

[{'score': 0.14473257958889008,
  'start': 43,
  'end': 56,
  'answer': 'United States',
  'context': "By total area (water as well as land), the United States is either slightly larger or smaller than the People's Republic of China, making it the world's third or fourth largest country. China and the United States are smaller than Russia and Canada in total area, but are larger than Brazil. By land area only (exclusive of waters), the United States is the world's third largest country, after Russia and China, with Canada in fourth. Whether the US or China is the third largest country by total area depends on two factors: (1) The validity of China's claim on Aksai Chin and Trans-Karakoram Tract. Both these territories are also claimed by India, so are not counted; and (2) How US calculates its own surface area. Since the initial publishing of the World Factbook, the CIA has updated the total area of United States a number of times."}]

In [52]:
question = "What is the oldest company in the world?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Nordisk Film company',
  'context': "It is the world's fourth oldest major film studio, after the "
             'renowned French studios Gaumont Film Company and Pathé, and the '
             'Danish Nordisk Film company.',
  'end': 152,
  'score': 6.705815280838578e-07,
  'start': 132}]


[{'score': 6.705815280838578e-07,
  'start': 132,
  'end': 152,
  'answer': 'Nordisk Film company',
  'context': "It is the world's fourth oldest major film studio, after the renowned French studios Gaumont Film Company and Pathé, and the Danish Nordisk Film company."}]

In [53]:
question = "What is the current biggest animal on earth?"
context = get_context(question, top_k=3)
extract_answer(question, context)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'answer': "Earth's current species range from 10 million to 14 million,",
  'context': 'Rapid environmental changes typically cause mass extinctions. '
             'More than 99 percent of all species, amounting to over five '
             'billion species, that ever lived on Earth are estimated to be '
             "extinct. Estimates on the number of Earth's current species "
             'range from 10 million to 14 million, of which about 1.2 million '
             'have been documented and over 86 percent have not yet been '
             'described. The total amount of related DNA base pairs on Earth '
             'is estimated at 5.0 x 1037, and weighs 50 billion tonnes. In '
             'comparison, the total mass of the biosphere has been estimated '
             'to be as much as 4 TtC (trillion tons of carbon).',
  'end': 280,
  'score': 0.0003270465531386435,
  'start': 220},
 {'answer': 'whales',
  'context': 'Mammals include the largest animals on the planet, the rorq

[{'score': 0.0003270465531386435,
  'start': 220,
  'end': 280,
  'answer': "Earth's current species range from 10 million to 14 million,",
  'context': "Rapid environmental changes typically cause mass extinctions. More than 99 percent of all species, amounting to over five billion species, that ever lived on Earth are estimated to be extinct. Estimates on the number of Earth's current species range from 10 million to 14 million, of which about 1.2 million have been documented and over 86 percent have not yet been described. The total amount of related DNA base pairs on Earth is estimated at 5.0 x 1037, and weighs 50 billion tonnes. In comparison, the total mass of the biosphere has been estimated to be as much as 4 TtC (trillion tons of carbon)."},
 {'score': 1.1971280855505029e-06,
  'start': 80,
  'end': 86,
  'answer': 'whales',
  'context': 'Mammals include the largest animals on the planet, the rorquals and other large whales, as well as some of the most intelligent, such as ele